In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import skmob
from skmob import preprocessing
from shapely.geometry import LineString
import folium

In [2]:
def create_plot(data):
    # Ambil baris pertama dari DataFrame
    first_row = data.iloc[0]

    # Ambil nilai latitude dan longitude dari baris pertama
    latitude = first_row['latitude']
    longitude = first_row['longitude']
    
    m = folium.Map(location=[latitude, longitude], zoom_start=25)

    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"User ID: {row['maid']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
        ).add_to(m)
    
    return m

In [3]:
def create_pivot(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df_pivot = df['maid'].groupby(df['timestamp'].dt.date).value_counts()
    pivot = df_pivot.unstack().fillna(0).astype(int)

    total_counts = pivot.sum(axis=0)
    sorted_columns = total_counts.sort_values(ascending=False).index
    pivot_sorted = pivot[sorted_columns]
    return pivot_sorted

In [4]:
def create_line(data):
    data = data.to_crs('EPSG:4326')
    data = data['geometry']
    # Buat peta dengan lokasi awal berdasarkan rata-rata koordinat dari data linestring
    avg_lat = data.apply(lambda x: x.centroid.y).mean()
    avg_lon = data.apply(lambda x: x.centroid.x).mean()
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

    # Tambahkan polyline untuk setiap linestring
    for linestring in data:
        coordinates = [(lat, lon) for lon, lat in linestring.coords]
        folium.PolyLine(locations=coordinates, color='purple', weight=8, opacity=0.7).add_to(m)

    # Tambahkan GeoJson dari data GeoPandas
    folium.GeoJson(data.to_json(), name='Garis Jalan').add_to(m)

    return m

## Import data

In [5]:
path = './filter2_malar_des.csv'
gps = pd.read_csv(path)

## Try to filtering

In [6]:
gps.head()

,maid,latitude,longitude,timestamp
0,0000db5b-e162-4730-a6c5-11f65b3872b7,-7.800905,110.366638,2021-12-21 14:01:13
1,0000db5b-e162-4730-a6c5-11f65b3872b7,-7.800905,110.366640,2021-12-21 21:07:15
2,0000db5b-e162-4730-a6c5-11f65b3872b7,-7.800845,110.366638,2021-12-23 11:27:57
3,0000db5b-e162-4730-a6c5-11f65b3872b7,-7.800845,110.366640,2021-12-23 11:31:27
4,00012afc-6daf-461f-96a8-181e5af69db9,-7.800291,110.364998,2021-12-01 17:54:11


In [43]:
# create_pivot(gps)

In [39]:
orang = gps[(gps['maid'] == 'd60a2068-d7ea-48a7-a363-e3e7e0bd02cb')]
orang.reset_index(inplace=True, drop=True)
len(orang)

72

### Hapus data yg memiliki jarak 1 jam dari record sebelumnya dan record setelahnya

In [54]:
def remove_records_hour(a, timedelta):
    a['timestamp'] = pd.to_datetime(a['timestamp'])
    
    diff_prev = a['timestamp'] - a['timestamp'].shift(1)
    diff_next = a['timestamp'].shift(-1) - a['timestamp']
        
    # Aturan untuk baris-baris lainnya
    to_keep = ((diff_prev <= pd.Timedelta(minutes=timedelta)) | (diff_next <= pd.Timedelta(minutes=timedelta)))

    # Ambil baris-baris yang ditandai untuk dipertahankan
    group_filtered = a[to_keep]
        
    group_filtered.reset_index(inplace=True, drop=True)
    
    return group_filtered

In [55]:
df_filtered_20m_1hour = remove_records_hour(orang, 1)

/tmp/ipykernel_17218/4211185109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['timestamp'] = pd.to_datetime(a['timestamp'])


In [56]:
df_filtered_20m_1hour

,maid,latitude,longitude,timestamp
0,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.794847,110.365608,2021-12-01 00:00:35
1,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.794847,110.365608,2021-12-01 00:01:27
2,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.805548,110.370445,2021-12-01 00:46:45
3,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.805548,110.370445,2021-12-01 00:47:16
4,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.791002,110.366051,2021-12-01 00:53:53
5,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.791002,110.366051,2021-12-01 00:54:21
6,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.803500,110.364601,2021-12-01 01:19:53
7,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.791010,110.366051,2021-12-01 01:20:51
8,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.791010,110.366051,2021-12-01 01:25:44
9,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.791002,110.366051,2021-12-01 01:26:40


In [38]:
orang

,maid,latitude,longitude,timestamp
0,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772810,110.357910,2021-12-29 00:00:37
1,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772810,110.357918,2021-12-29 00:00:38
2,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772802,110.357918,2021-12-29 00:00:47
3,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772802,110.357918,2021-12-29 00:00:48
4,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772760,110.357780,2021-12-29 00:02:09
...,...,...,...,...
1424,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772757,110.357857,2021-12-31 22:51:32
1425,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772770,110.357860,2021-12-31 22:51:41
1426,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772770,110.357849,2021-12-31 22:51:42
1427,9a63fd0e-d363-478a-8136-99b1eb82693f,-7.772760,110.357849,2021-12-31 22:59:33
